Run: docker compose -f graph-docker-compose.yml up -d

In [1]:
from neo4j import GraphDatabase
# from tabulate import tabulate
import pandas as pd

In [2]:
# Połączenie z Neo4j
URI = "bolt://neo4j_nosql_lab:7687"
USERNAME = "neo4j"
PASSWORD = "test1234"  # upewnij się, że to pasuje do graph-docker-compose.yml

In [ ]:
def load_sample_data(tx):
    tx.run("""
       // FILMY
       CREATE (matrix:Movie {title: 'The Matrix', released: 1999}),
       (johnwick:Movie {title: 'John Wick', released: 2014}),
       (inception:Movie {title: 'Inception', released: 2010}),
       (tenet:Movie {title: 'Tenet', released: 2020}),

       // OSOBY
       (keanu:Person {name: 'Keanu Reeves'}),
       (laurence:Person {name: 'Laurence Fishburne'}),
       (lana:Person {name: 'Lana Wachowski'}),
       (chad:Person {name: 'Chad Stahelski'}),
       (nolan:Person {name: 'Christopher Nolan'}),
       (leo:Person {name: 'Leonardo DiCaprio'}),
       (elizabeth:Person {name: 'Elizabeth Debicki'}),

       // GATUNKI
       (scifi:Genre {name: 'Sci-Fi'}),
       (action:Genre {name: 'Action'}),
       (thriller:Genre {name: 'Thriller'}),

       // OCENY
       (user1:User {name: 'student1'}),
       (user2:User {name: 'student2'}),

       // RELACJE FILM - OSOBY
       (keanu)-[:ACTED_IN]->(matrix),
       (laurence)-[:ACTED_IN]->(matrix),
       (lana)-[:DIRECTED]->(matrix),

       (keanu)-[:ACTED_IN]->(johnwick),
       (chad)-[:DIRECTED]->(johnwick),

       (leo)-[:ACTED_IN]->(inception),
       (nolan)-[:DIRECTED]->(inception),

       (elizabeth)-[:ACTED_IN]->(tenet),
       (nolan)-[:DIRECTED]->(tenet),

       // RELACJE FILM - GATUNEK
       (matrix)-[:HAS_GENRE]->(scifi),
       (matrix)-[:HAS_GENRE]->(action),
       (johnwick)-[:HAS_GENRE]->(action),
       (inception)-[:HAS_GENRE]->(scifi),
       (inception)-[:HAS_GENRE]->(thriller),
       (tenet)-[:HAS_GENRE]->(scifi),
       (tenet)-[:HAS_GENRE]->(action),

       // RELACJE SPOŁECZNE
       (keanu)-[:FOLLOWS]->(laurence),
       (lana)-[:FOLLOWS]->(nolan),
       (leo)-[:FOLLOWS]->(keanu),

       // RELACJE OCENIANIA
       (user1)-[:RATED {score: 5}]->(matrix),
       (user1)-[:RATED {score: 4}]->(johnwick),
       (user2)-[:RATED {score: 3}]->(tenet),
       (user2)-[:RATED {score: 4}]->(inception)
    """)

driver = GraphDatabase.driver(URI, auth=(USERNAME, PASSWORD))

with driver.session() as session:
    session.write_transaction(load_sample_data)

print("Dane zostały załadowane!")

driver.close()


/tmp/ipykernel_3717/3669902964.py:68: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(load_sample_data)


Dane zostały załadowane!


In [1]:
def run_cypher(query: str):
    with driver.session() as session:
        result = session.run(query)
        records = list(result)
        if not records:
            print("🔍 Brak wyników.")
            return
        # Przekształcenie do DataFrame
        df = pd.DataFrame([r.data() for r in records])
        return df

print("Połączono z Neo4j. Użyj funkcji `run_cypher('TWOJE_ZAPYTANIE')` aby wykonać zapytanie.")

def qexec(query: str):
    driver = GraphDatabase.driver(URI, auth=(USERNAME, PASSWORD))
    
    df = run_cypher(query)
    
    driver.close()
    
    return df

Połączono z Neo4j. Użyj funkcji `run_cypher('TWOJE_ZAPYTANIE')` aby wykonać zapytanie.


In [21]:
# PODSTAWY ZAPYTAŃ

# Wszyscy aktorzy (osoby, które zagrały w filmach)
q1 = """
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
RETURN p.name, m.title
"""

# Wyświetl wszystkich aktorów w bazie (czyli osoby, które grały w filmach).
q2 = """
MATCH (p:Person)-[:ACTED_IN]->(:Movie)
RETURN DISTINCT p.name;
"""

# Wyświetl wszystkie filmy wraz z datą wydania.
q3 = """
MATCH (m:Movie)
RETURN m.title, m.released;
"""

# Znajdź wszystkie osoby, które grały w filmie „The Matrix”.
q4 = """
MATCH (p:Person)-[:ACTED_IN]->(m:Movie {title: 'The Matrix'})
RETURN p.name;
"""

# Kto wyreżyserował film „Inception”?
q5 = """
MATCH (p:Person)-[:DIRECTED]->(m:Movie {title: "Inception"})
RETURN p.name AS Director;
"""

# Jakie gatunki ma film „Tenet”?
q6 = """
MATCH (m:Movie {title: "Tenet"})-[:HAS_GENRE]->(g:Genre)
RETURN g.name AS Genre;
"""

# RELACJE

# Kto kogo obserwuje
q7 = """
MATCH (a:Person)-[:FOLLOWS]->(b:Person)
RETURN a.name AS Follower, b.name AS Followed;
"""

# Kto obserwuje Keanu Reevesa
q8 = """
MATCH (p:Person)-[:FOLLOWS]->(:Person {name: "Keanu Reeves"})
RETURN p.name;
"""

# Najkrótsza ścieżka między Leonardo DiCaprio a Laurence Fishburne
q9 = """
MATCH path = shortestPath(
  (a:Person {name: "Leonardo DiCaprio"})-[:FOLLOWS*..5]-(b:Person {name: "Laurence Fishburne"})
)
RETURN path;
"""

# AGREGACJE I STATYSTYKI

# Aktorzy grający w więcej niż jednym filmie
q10 = """
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WITH p, count(m) AS appearances
WHERE appearances > 1
RETURN p.name, appearances;
"""

# Reżyserzy według liczby filmów
q11 = """
MATCH (p:Person)-[:DIRECTED]->(m:Movie)
RETURN p.name AS Director, count(m) AS MoviesDirected
ORDER BY MoviesDirected DESC;
"""

# Średnia ocen filmów
q12 = """
MATCH (u:User)-[r:RATED]->(m:Movie)
RETURN m.title, round(avg(r.score), 2) AS AverageRating
ORDER BY AverageRating DESC;
"""

# ZŁOŻONE WZORCE

# Filmy Sci-Fi po 2010 roku
q13 = """
MATCH (m:Movie)-[:HAS_GENRE]->(g:Genre {name: "Sci-Fi"})
WHERE m.released > 2010
RETURN m.title, m.released;
"""

# Aktorzy w filmach ocenionych na 5
q14 = """
MATCH (u:User)-[r:RATED {score: 5}]->(m:Movie)<-[:ACTED_IN]-(p:Person)
RETURN DISTINCT p.name, m.title;
"""

# Reżyserzy, którzy obserwują innych reżyserów
q15 = """
MATCH (a:Person)-[:FOLLOWS]->(b:Person)
WHERE (a)-[:DIRECTED]->() AND (b)-[:DIRECTED]->()
RETURN a.name AS Reżyser, b.name AS Obserwowany;
"""

# Filmy z Keanu Reevesem i gatunkiem "Action"
q16 = """
MATCH (p:Person {name: "Keanu Reeves"})-[:ACTED_IN]->(m:Movie)-[:HAS_GENRE]->(g:Genre {name: "Action"})
RETURN p.name, m.title;
"""

# UPDATE WŁAŚCIWOŚCI

# Zmieniamy imię osoby i dodajemy nową właściwość „urodzony”
q17 = """
MATCH (p:Person {name: "Keanu Reeves"})
SET p.name = "Keanu Charles Reeves",
    p.born = 1964
RETURN p;
"""

# Dodajemy nową etykietę Film i usuwamy starą Movie, a także dodajemy flagę updated = true
q18 = """
MATCH (m:Movie {title: "John Wick"})
SET m:Film, m.updated = true
REMOVE m:Movie
RETURN m;
"""

# Użytkownik student1 zmienia ocenę filmu „The Matrix” z 5 na 4
q19 = """
MATCH (:User {name: "student1"})-[r:RATED]->(:Movie {title: "The Matrix"})
SET r.score = 4,
    r.updatedAt = date()
RETURN r;
"""

# Usunięcie właściwości born z osoby Keanu Reevesa
q20 = """
MATCH (p:Person {name: "Keanu Charles Reeves"})
REMOVE p.born
RETURN p;
"""

# Usunięcie pola updatedAt z relacji RATED wystawionej przez student1
q21 = """
MATCH (:User {name: "student1"})-[r:RATED]->(:Movie {title: "The Matrix"})
REMOVE r.updatedAt
RETURN r;
"""

# Ponieważ Neo4j nie pozwala zmieniać typu relacji bezpośrednio, robimy to w 3 krokach: kopiujemy dane, tworzymy nową relację, usuwamy starą
q22 = """
MATCH (a:Person {name: "Lana Wachowski"})-[r:FOLLOWS]->(b:Person {name: "Christopher Nolan"})
CREATE (a)-[:MENTORS {since: 2020}]->(b)
DELETE r;
"""

# Czyszczenie całej bazy
q23 = """
MATCH (n)
DETACH DELETE n;
"""

In [ ]:
# qexec(q16)
qexec(q23)

/tmp/ipykernel_162/1749906726.py:2: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:


,p.name,m.title
0,Keanu Reeves,The Matrix
1,Keanu Reeves,John Wick
